In [1]:
import glob, os.path, re, json
from sklearn import svm

In [23]:
def check_freq(fname):
    with open(fname,'r',encoding='utf-8') as f: #f라는 통로를 통해 fname에 해당하는 파일을 utf-8로 인코딩해서 가져옴
        text = f.read() #주로 readline을 이용해서 읽지만 여기선 글자 하나하나 읽을 것이기 때문에 줄바꿈이 의미가 없어서 read사용
        text = text.lower() #a-z 빈도 셀 것이기 때문에 대문자로 되어있는것들 소문자로 처리
    cnt = [0 for i in range(0,26)] #a-z빈도가 들어갈 기본 틀 생성 [0,0,0,…,0] (26개)
    code_a = ord('a') #'a' 아스키 코드처리 = 97
    code_z = ord('z') #'z' 아스키 코드처리 = 122
    for ch in text:   #text에서 글자 1개씩(ch) 가져옴
        n = ord(ch)   #읽어온 글자 아스키 코드처리
        if code_a <= n <= code_z:  #n이 a-z에 해당하는 것만 if문 수행 (특수문자,숫자 등 filter효과)
             cnt[n - code_a] += 1   #n값을 cnt 내부의 인덱스화
                                    #예를 들어 ch가 'a'일 때, n은 97로 code_a와 같은값이 되어 cnt[0]이 되고
                                    #cnt[0]자리에 1 누적되므로 각 알파벳이 나올때마다 해당하는 자리에 누적되어
                                    #a-z까지 각각의 알파벳 빈도를 담은 리스트를 만들 수 있음
                                    #ex) [349, 59, 210, 212, 484,…, 45, 65, 3, 92, 2]
    
    #정규화
    #각 txt파일마다 글자수가 차이나므로, 비교를 위해 빈도대신 비율값 사용
    total = sum(cnt) #한 파일에서 나온 text 수의 총합
    freq = list(map(lambda n:n/total,cnt)) #각 알파벳 빈도/text수 총합 >> 비율화
                                            #n은 cnt의 원소, map: cnt에서 원소n을 하나씩 꺼내서 함수 반복수행
    
    name = os.path.basename(fname)  #basename은 fname에서 경로를 제외한 순수 제목만 뽑아주는 함수
                                    #ex) en-1.txt
    label = re.match(r'^[a-z]{2,}',name).group() #정규표현식을 사용해, 앞에 두글자만 추출
                                                 #group()은 match된 문자열을 돌려주는 함수
    
    return freq, label    #값을 두 개 출력하는것 처럼 보이지만 사실상 (freq,label)의 튜플형식으로 추출함


def load_files(path):
    
    freqs = []
    labels = []
    file_lists = glob.glob(path)  #glob모듈의 glob함수. 디렉터리 내의 모든 파일을 리스트로 돌려주는 함수
                                  #'*', '?' 등 메타문자를 써서 원하는 파일만 불러올 수 있음
                                  #ex) './lang/train/*.txt'는 해당 경로 내의 txt파일을 모두 불러옴
    for fname in file_lists:     
        r = check_freq(fname)
        freqs.append(r[0])        #r[0]은 freq. freqs에 하나씩 담음 (행으로)
        labels.append(r[1])       #r[1]은 label. labels에 하나씩 담음 (행으로)
    return {'freqs':freqs, 'labels':labels} #위에서 하나씩 담아 만든 리스트를 딕셔너리 형태로 반환
                                             #딕셔너리 형식은 json형식과 완벽히 호환가능

### main 부분
data = load_files('./lang/train/*.txt')
test = load_files('./lang/test/*.txt')

with open('./lang/freq2.json','w',encoding='utf-8') as fp:  #freq2.json파일을 생성해 저장할 것. fp 통로생성
    json.dump([data,test],fp,indent=4) #fp객체. 즉, freq2.json파일에 [data,test]를 리스트로 담아서 저장할 것
                                       #dump는 오픈한 파일에 바로 써서 저장함
                                       #dumps는  파이썬 내에서 사용하기 위해 json 형식으로 string으로 반환
                                       #indent는 가독성을 위한 들여쓰기 부분 없어도 무방함